# Taller de NLP: Word2Vec

In [ ]:
#%load_ext watermark
#%watermark -a 'Santiago Alferez' --iversions

In [ ]:
numpy     : 1.21.2
pandas    : 1.3.5
re        : 2.2.1
nltk      : 3.6.7
sklearn   : 1.0.2
matplotlib: 3.5.0
scipy     : 1.7.3
gensim    : 4.0.1

SyntaxError: ignored

In [ ]:
import pandas as pd
import nltk

Para este taller deberás disponer de algunas librerías como scikit-learn, NLTK, y GenSim. Se recomienda revisar la [documentación de GenSim](https://radimrehurek.com/gensim/auto_examples/index.html#documentation).

## Cargando un modelo  en GenSim y análisis
A continuación cargaremos un modelo que no pesa tanto `glove-twitter-50`. Hay modelos más completos y con mayor número de dimensiones en este [link](https://github.com/RaRe-Technologies/gensim-data)

In [ ]:
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api

In [ ]:
wv = api.load("glove-twitter-50")

[==================================================] 100.0% 199.5/199.5MB downloaded


Una operación común es recuperar el vocabulario de un modelo. Eso es trivial:

In [ ]:
!pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 56.4 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
for index, word in enumerate(wv.index_to_key):
    if index == 20:
        break
    print(f"palabra #{index}/{len(wv.index_to_key)} es {word}")

palabra #0/1193514 es <user>
palabra #1/1193514 es .
palabra #2/1193514 es :
palabra #3/1193514 es rt
palabra #4/1193514 es ,
palabra #5/1193514 es <repeat>
palabra #6/1193514 es <hashtag>
palabra #7/1193514 es <number>
palabra #8/1193514 es <url>
palabra #9/1193514 es !
palabra #10/1193514 es i
palabra #11/1193514 es a
palabra #12/1193514 es "
palabra #13/1193514 es the
palabra #14/1193514 es ?
palabra #15/1193514 es you
palabra #16/1193514 es to
palabra #17/1193514 es (
palabra #18/1193514 es <allcaps>
palabra #19/1193514 es <elong>


In [ ]:
vec_king = wv['king']

In [ ]:
vec_king

Desafortunadamente, el modelo no puede inferir vectores para palabras desconocidas. Esta es una limitación de Word2Vec: si esta limitación le importa, consulte el modelo FastText.

In [ ]:
wv["cielo"]

Continuando, Word2Vec admite varias tareas de similitud de palabras listas para usar. Puedes ver cómo la similitud (¿Que similitud será?) disminuye intuitivamente a medida que las palabras se vuelven cada vez menos similares.

In [ ]:
pairs = [
    ('gato', 'perro'),   
    ('gato', 'tigre'),   
    ('gato', 'rana'),  
    ('gato', 'nube'),   
    ('gato', 'politica'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

Podemos encontrar las palabras más similares de acuerdo a una medida de similaridad ([la similaridad del coseno](https://en.wikipedia.org/wiki/Cosine_similarity)):

In [ ]:
pd.DataFrame(wv.most_similar(positive=['carro', 'camión'], topn=5), columns=["palabra", "similaridad"])

¿Cuál de estos no pertenece a la secuencia?

In [ ]:
print(wv.doesnt_match(['fuego', 'agua', 'tierra', 'mar', 'aire', 'carro']))

### Analogias

Se ha demostrado que los vectores de palabras *a veces* exhiben la capacidad de resolver analogías.

Como ejemplo, para la analogía "hombre: rey :: mujer: x" (léase: el hombre es al rey como la mujer es a x), ¿qué es x?

En la celda a continuación, se muestra cómo usar vectores de palabras para encontrar x usando la función `most_similar` de la [documentacion de GenSim](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similares). La función encuentra palabras que son más similares a las palabras en la lista "positiva" y más diferentes de las palabras en la lista "negativa" (mientras omite las palabras de entrada, que a menudo son las más similares). La respuesta a la analogía tendrá la mayor similitud del coseno (mayor valor numérico devuelto).

In [ ]:
# esta es la analogia -- hombre : rey :: mujer : x
wv.most_similar(positive=['mujer', 'rey'], negative=['hombre'])

#### Ejercicio 1
Cree una función denominada `analogia(a,b,c)` donde determine la analogía *a* : *b* :: *c* : *d*, es decir retorne a *d*, como en el ejemplo anterior. Pruebe la función con 5 analogías que se le ocurra (en inglés o en españól, pero no combinadas)

In [ ]:
def analogia(a,b,c):
  lista = wv.most_similar(positive=[a,b],negative=[c])
  return lista

In [ ]:
analogia('guerra','arma','argumento')

In [ ]:
analogia('blanco','dia','negro')

In [ ]:
analogia('cafe','caliente','frio')

In [ ]:
analogia('estudiante', 'examenes', 'profesor')

In [ ]:
analogia('leer', 'aprender', 'correr')

#### Ejercicio 2
Haga un listado de unas 20 palabras y grafiquelas en dos dimensiones (colocando su texto) junto con unas 200 palabras muestreadas aleatoriamente (sin texto). Use para reducir la dimensión PCA. Repita lo mismo usando UMAP.

In [ ]:
from sklearn.decomposition import PCA

def f_get_words(text_, cat_palabras):
  global wv
  word_similitud = wv.similar_by_word(text_, topn=cat_palabras)
  similar_words = list()

  for i in range(len(word_similitud)):
    if word_similitud[i][1]>0:
      temp=word_similitud[i][0]
      similar_words.append(temp)

  vectors = wv[similar_words]

  return vectors, similar_words


para_pca, nombres_para_pca =  f_get_words(text_= 'perro', cat_palabras=200)
para_pintar, nombres_para_pintar =  f_get_words(text_= 'razas', cat_palabras=20)


#reducir dimensionalidad por componentes con PCA.
reduccion =  PCA(random_state=0, n_components  = 2).fit(para_pca)

#transformando los datos de los pntos
puntos_plot = reduccion.transform(para_pintar)

# organizando para pintar via sns
base=pd.DataFrame()
base["primer_componente"]=puntos_plot[:,0]
base["segundo_componente"]=puntos_plot[:,1]
base["nombres"]=nombres_para_pintar
#display(base)

import seaborn as sns #visializacoines
graf1=sns.scatterplot(x="primer_componente", y="segundo_componente", data=base)

for line in range(0,base.shape[0]):
     graf1.text(base.primer_componente[line]+0.01, base.segundo_componente[line], 
     base.nombres[line], horizontalalignment='left', 
     size='medium', color='black', weight='semibold')

In [ ]:
# USANDO umap

!pip install umap-learn

import umap.umap_ as umap

reducer = umap.UMAP(random_state=0)

embedding = reducer.fit(para_pca)

points_plot = embedding.transform(para_pintar)

# organizando para pintar via sns
base=pd.DataFrame()
base["primer_componente"]=points_plot[:,0]
base["segundo_componente"]=points_plot[:,1]
base["nombres"]=nombres_para_pintar
#display(base)

import seaborn as sns #visializacoines
graf1=sns.scatterplot(x="primer_componente", y="segundo_componente", data=base)

for line in range(0,base.shape[0]):
     graf1.text(base.primer_componente[line]+0.01, base.segundo_componente[line], 
     base.nombres[line], horizontalalignment='left', 
     size='medium', color='black', weight='semibold')

-----

GenSim permite acceder a las palabras mediante diferentes formas sobre un objeto `KeyedVectors` (el wv de antes es uno). `.index_to_key`produce una lista con el vocabulario de forma ordenada, mientras `.key_to_index` produce un diccionario de la forma {palabra: index}.

In [ ]:
print(wv.index_to_key[100:120])

## Entrene un nuevo modelo sobre un corpus

Trabajaremos de nuevo con el dataset de `progressive-tweet-sentiment.csv`, el cuál es un dataset pequeño que nos facilitará probar Word2vec. Sin embargo, los resultados pueden no ser tan buenos, dado que Word2vec es más potente cuando el corpus es más grande.

`progressive-tweet-sentiment`  tiene algunos tweets recopilados y categorizados en 4 clases: 'Legalization of Abortion', 'Hillary Clinton', 'Feminist Movement', 'Atheism'. 

In [ ]:
df = pd.read_csv("/content/progressive-tweet-sentiment.csv",encoding='latin-1')

In [ ]:
df = df[["target", "tweet"]]
df.head()

#### Ejercicio 3. 
Realice procedimientos para preprocesar texto cómo: tokenizar, eliminar stopwords, stemming y lemmatization. Al final, el resultado de dicho procesamiento debe ser un texto (no una lista de palabras). Sugerencia: inicie y finalice con métodos de strings de python como `.join()` o `.split()`. (Esto ya lo han hecho en el taller anterior).

Nota: En NLP, a menudo agregamos tokens <START> y <END> para representar el principio y el final de oraciones, párrafos o documentos. En este caso, pueden colocar tokens `<START>` y `<END>` encapsulando cada documento, por ejemplo, "<START> All that glitters not gold <END>", y se puede incluir los tokens en el corpus completo. No es necesario hacer esto para el ejercicio, pero sería interesante.

In [ ]:
import re
def limpieza(text):
  #text=re.sub(r'@_[A-Za-z0-9]+_','',text) #Remover @_menciones_
  #text=re.sub(r'@[A-Za-z0-9]+_[A-Za-z0-9]+','',text) #Remover @menciones_
  #text=re.sub(r'@[A-Za-z0-9]+','',text) #Remover @menciones
  text=re.sub(r'https?:\/\/\S+','',text) #Remover Hypervinculos
  text=re.sub(r'RT[\s]+','',text) #Remover Retweets
  #text=re.sub(r'#[A-Za-z0-9]+','',text) #Remover "#"
  #text=re.sub(r'[0-9]','',text) #remover numeros
  text=re.sub(r'[^\w\s]',"",text) #remover signos de puntuacion
  text=re.sub(r'\n|\t',' ',text) #remover saltos de linea
  text=re.sub(r'\@',' ',text) #remover "@"
  text=re.sub(r'[\s]+',' ',text) #reemplazar espacios dobles por espacion sencillos
  text=re.sub(r"^[\s]",' ',text) #eliminar espacios al inicio de cada tweet
  text=text.lower() #todo a minuscula
  return text

In [ ]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
#limpieza inicial de los tweetseza)
# Excluir stopwords
df['text_clean']=df["tweet"].apply(limpieza)
df['text_clean'] = df["text_clean"].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))
df['text_clean'] 

In [ ]:
#Debe quedar algo asi:

### Entrada al modelo
Para entrenar un modelo en GenSim es importante adecuar el texto a una lista de sentencias (y cada sentencia una lista de tokens). Para corpus muy grandes, es mejor crear un iterador (una función que extraiga documento a documento para evitar llenar la memoria). 

Por ejemplo las tres primeras sentencias del dataset son:

In [ ]:
df['text_clean']

In [ ]:
listado = []
for i in range(len(df)):
  listado.append (list(df['text_clean'][i].split(' ')))

In [ ]:
print(listado)

In [ ]:
len(listado)

#### Ejercicio 4
Entrene un modelo usando `gensim.models.word2vec.Word2Vec`, partiendo de la siguiente configuración de parámetros.

In [ ]:
b=1

In [ ]:
vector_size = 100 # numero de elementos del vector que representa la palabra
min_count =3 # Ignores all words with total frequency lower than this. 
workers = 5 # numero de cpu cores
sg = 0 # 0: CBOW, 1: skip-gram
window = 6 # Tamano de la ventana de contexto
sample = 1e-3 # tasa de submuestreo para terminos frecuentes

In [ ]:
Word2Vec?

In [ ]:
from gensim.models import Word2Vec

In [ ]:
model = Word2Vec(listado, size=vector_size,min_count=min_count, workers=workers,sg=sg,window=window,sample=sample)

# Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)


#### Ejercicio 5
Ahora ha obtenido un modelo de Word2vec en el cual tiene una representación embebida de cada palabra. Esta representación la puede extraer para cada palabra usando `model.wv.get_vector(palabra)`. Sin embargo, ¿Qué representación podemos obtener para cada tweet (documento o sentencia) a partir de todas las palabras? 

Cree una función que extraiga para cada tweet un representación vectorial única (un vector) y añada una nueva columna con esta representación. Sugerencia: una suma (pero será lo mejor?). El resultado es algo similar a la celda siguiente:

In [ ]:
#funcion para calcular el promedio de los vectores de cada token por tweet
import numpy as np
tweet_vector=[]
for i in range(len(listado)):
  v1=[0 for i in range(vector_size)]
  for j in range(len(listado[i])):
    if listado[i][j] in model.wv:
      v2=model.wv.get_vector(listado[i][j])
      v2=1/np.log(1+v2)
      v1=[v1[k] + v2[k] for k in range(len(v1))]
  v1=[v1[k]/len(v1) for k in range(len(v1))]
  tweet_vector.append(v1)

#tweet_vector

df2=df[:]
df2
df2['vectors']=tweet_vector
df2.head(n=5)
#len(tweet_vector)


#df2.vectors[0:10]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,target,tweet,text_clean,vectors
0,Legalization of Abortion,Thank you for another day of life Lord. #Chris...,thank another day life lord christian catholic...,"[-82.39495712280274, 17.74701385498047, -64.96..."
1,Legalization of Abortion,@rosaryrevival Lovely to use Glorious Mysterie...,rosaryrevival lovely use glorious mysteries ea...,"[-16.865619506835937, -106.11549591064453, -34..."
2,Legalization of Abortion,@Niall250 good thing is that #DUP have consist...,niall250 good thing dup consistently said murd...,"[-160.62926284790038, 3.9710247802734373, 8.83..."
3,Legalization of Abortion,"So, you tell me... is murder okay if the victi...",tell murder okay victim mentally disabled,"[-89.69256469726562, -6.972107086181641, 3.402..."
4,Legalization of Abortion,@HillaryClinton Don't you mean to say (all chi...,hillaryclinton dont mean say children deserve ...,"[12.638732604980468, -2.2604055786132813, 106...."


#### Ejercicio 6.
Lo que se ha generado antes son unas determinadas variables, para cada tweet, úselas para realizar una clasificación. Para esto divida en un conjunto de entrenamiento y uno de prueba (20%) con una semilla fija (42) y construya un clasificador (recomendado Random Forest). Evalúe el desempeño del clasificador en el conjunto de prueba con el accuracy.  El accuracy debe estar alrededor del 70%. Para lograrlo, tendrá que cambiar la longitud del vector, el ancho de la ventana, si es skip-gram o CBOW, el método de obtención de un sólo documento, etc.


In [ ]:
#convertir vectores en dataframe
X=pd.DataFrame(tweet_vector)
X

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-82.394957,17.747014,-64.963592,27.101215,-12.252192,41.174217,141.665765,-305.211515,-16.676145,183.931009,...,11.259987,36.712780,8.873947,-29.844742,-19.655383,16.956199,33.939394,-14.528294,-26.529897,-40.935954
1,-16.865620,-106.115496,-34.669024,8.767858,-4.480404,-35.589104,-33.896736,-4.147153,-3.876323,22.127322,...,-117.657072,31.434496,32.314885,2.879260,60.842038,-3.819093,0.802055,-9.498622,6.029724,-2.693962
2,-160.629263,3.971025,8.836054,-76.970793,21.098858,-30.070327,-54.966711,-34.515130,-30.361694,16.701591,...,-58.735923,9.002352,-18.750515,-25.426743,9.098057,-41.536447,-35.850288,29.277635,45.060954,-26.414734
3,-89.692565,-6.972107,3.402935,-12.327951,-28.517099,-2.074270,-149.382354,-13.449227,-10.591844,5.105086,...,5.065800,142.803172,-30.065370,-6.399946,4.263249,-0.442385,-15.047195,-18.943108,-28.201361,-26.663290
4,12.638733,-2.260406,106.529258,16.671176,41.239812,-62.002429,45.582859,17.781214,-1.732364,-5.639399,...,-10.808809,-10.684872,21.167086,170.157104,-10.504815,69.872334,-19.981788,-22.274722,8.691012,34.215763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154,13.223297,-2.552437,69.821738,-48.335599,62.377303,10.085500,-6.456139,24.778224,8.184739,-7.015747,...,24.861034,25.628554,-4.216089,27.925299,-2.703618,-5.727675,10.150323,-4.956790,-6.866812,-21.061949
1155,-0.953879,4.093142,10.464818,65.929048,-21.459181,4.836263,-7.948669,-0.136393,11.810644,-3.273144,...,5.230290,7.589268,-6.484727,-5.786966,64.554244,-8.568740,2.677154,-14.893141,89.020586,4.147711
1156,1.605782,-16.159986,7.639832,4.196319,-12.736492,4.506175,-2.484493,0.698669,-8.607734,-69.707010,...,-3.512492,0.205059,-3.297831,-6.975601,-59.020117,-3.787443,14.592027,21.793489,-8.657454,3.132642
1157,15.020533,-11.134451,15.935916,-15.313368,-28.040249,32.794753,6.205260,81.617057,-38.460098,-109.461796,...,7.676042,7.707631,-37.811305,85.868546,7.241477,34.288959,45.861973,-12.940251,-15.503968,-92.163583


In [ ]:
#obtener variable objetivo
y=pd.DataFrame(df2['target'])

In [ ]:
#separar dataset
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42) #tomar el 20% de los datos para validacion y definir la semilla en cero

In [ ]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
58,14.689934,31.314077,587.815862,-3.634257,-4.083807,-2.174355,1.959088,28.777176,-12.624671,17.942947,...,-18.266169,5.340341,12.634928,-32.271950,1.415777,-8.230480,-14.984251,7.540381,17.079293,-4.059496
333,-12.287529,12.142996,-107.623623,75.938946,-33.527145,43.069105,-97.236800,-182.362806,-59.478886,-28.899213,...,22.119186,3.222198,-35.910028,337.739881,14.184102,-1.865634,23.205483,-590.537049,56.056887,25.372918
332,-0.209439,3.190809,29.840580,35.183966,-8.697695,15.137548,-19.781899,-7.464166,1.650235,3.593336,...,12.705203,22.638451,-61.802587,164.978384,14.294694,-18.726506,13.857526,14.881923,-11.490167,2.883531
1065,7.219235,21.429429,0.093966,-22.746563,-44.664499,34.449047,-33.287779,30.535432,1.651443,15.664119,...,-2.967630,-16.340436,-1.356445,-20.196971,-6.117672,-15.507907,-16.998898,4.329723,-8.244011,1.543265
63,-7.166800,16.821356,26.041309,36.352217,34.168748,51.327075,-60.402205,68.968360,37.049200,8.139987,...,45.378235,3.082944,343.807268,45.688127,37.376707,-83.751136,61.882609,-23.513058,-1.316724,100.688365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,21.893614,35.478718,23.908988,31.787283,-26.887764,10.676749,43.040346,35.715576,35.537112,-8.671015,...,0.052516,11.756974,333.736482,-26.540441,119.783354,10.332289,-28.095857,437.092250,22.421526,94.724772
1095,-0.738249,4.532352,-21.965329,-2.378147,4.980123,10.301323,6.119273,-4.888874,-34.786157,-15.775530,...,3.354203,-15.292710,-19.168978,85.628882,-618.843982,9.969806,25.885599,-65.099606,63.476018,-6.187177
1130,-102.968882,151.503009,-783.937752,19.792426,5.201315,-34.501523,-12.440539,52.281354,40.843122,30.276867,...,12.133154,19.253254,10.773166,-28.413877,53.588727,11.446389,22.885034,-33.696458,-0.251941,-501.549083
860,-4.829867,8.150327,36.967912,13.502261,-0.733560,53.293233,27.296023,23.306299,-2585.407513,-6.456519,...,10.903937,-216.716663,-5.126592,16.417464,11.428832,-6.710279,7.578374,37.346935,2.926579,-62.008938


In [ ]:
#definir semilla
import random
random.seed(10)
#Importar modelo Ramdom Forest
from sklearn.ensemble import RandomForestClassifier
#Crear clasificador
clf=RandomForestClassifier(n_estimators=200,random_state=1000)
#Entrenar Modelo
clf.fit(X_train,y_train)
#predecir
y_pred=clf.predict(X_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


In [ ]:
#Importar metricas scikit-learn 
from sklearn import metrics
# calcular el "accuracy"
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6681034482758621
